# **特徵工程與多項式迴歸**

![](./images/C1_W2_Lab07_FeatureEngLecture.PNG)


## 目標
在本實驗中，你將：
- 探索「特徵工程」與「多項式迴歸」，學習如何利用線性迴歸的機制來擬合非常複雜、甚至高度非線性的函數。


## 工具
你將使用在前幾個實驗中撰寫的函式，以及 `matplotlib` 與 `NumPy` 等套件。 

In [ ]:
import sys, os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

#region 匯入資料
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None

def import_data_from_github():
    import os, urllib.request, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    #shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_multi.py", f"{UTILS_DIR}/lab_utils_multi.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")

repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    
os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.lab_utils_multi import zscore_normalize_features, run_gradient_descent_feng
np.set_printoptions(precision=2) 
plt.style.use('utils/deeplearning.mplstyle')

print("✅ 匯入模組及設定繪圖樣式")

#endregion 匯入資料


<br>

<a name='FeatureEng'></a>
# 特徵工程與多項式迴歸概觀（Overview）

在最基本的形式下，線性迴歸可以用來建立如下的模型：
$$f_{\mathbf{w},b} = w_0x_0 + w_1x_1+ ... + w_{n-1}x_{n-1} + b \tag{1}$$ 
那如果你的特徵／資料本身是非線性的，或是多個特徵的組合呢？例如：房價通常不會隨著室內面積呈現完全線性的關係，而是會對非常小或非常大的房子有額外的懲罰，產生如上圖所示的彎曲曲線。我們要如何利用「線性迴歸」這套機制來擬合這樣的曲線？
回想一下，我們手上的「機制」，就是可以調整 (1) 中的參數 $\mathbf{w}$、$\mathbf{b}$，讓這個方程式盡量貼合訓練資料。不過，無論怎麼調整 (1) 式中的 $\mathbf{w}$ 與 $\mathbf{b}$，都沒有辦法完美貼合一條真正的非線性曲線。


<a name='PolynomialFeatures'></a>
## 多項式特徵（Polynomial Features）

前面我們考慮的是資料本身是非線性的情況。現在試著用目前已學會的工具來擬合一條非線性曲線。我們從一個簡單的二次函數開始：$y = 1+x^2$。

你對這裡要用到的所有函式應該都很熟悉，它們都可以在 `lab_utils.py` 檔案中找到說明。我們會使用 [`np.c_[..]`](https://numpy.org/doc/stable/reference/generated/numpy.c_.html)，這是 NumPy 用來在欄方向（column）做串接的工具。

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = 1 + x**2
X = x.reshape(-1, 1)

model_w,model_b = run_gradient_descent_feng(X,y,iterations=1000, alpha = 1e-2)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("no feature engineering")
plt.plot(x,X@model_w + model_b, label="Predicted Value");  plt.xlabel("X"); plt.ylabel("y"); plt.legend(); plt.show()

如同預期，模型的擬合效果並不好。我們其實需要的是類似 $y = w_0x_0^2 + b$ 的形式，也就是加入一個**多項式特徵**。
要做到這件事，可以藉由修改「輸入資料」來「設計（engineer）」出所需的特徵。如果將原始的 $x$ 換成平方後的 $x^2$ 作為輸入，就能得到 $y = w_0x_0^2 + b$ 的模型。來試試看吧：在下面程式中，將 `X` 換成 `X**2`。

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = 1 + x**2

# Engineer features 
X = x**2      #<-- added engineered feature

In [ ]:
X = X.reshape(-1, 1)  #X should be a 2-D Matrix
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha = 1e-5)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("Added x**2 feature")
plt.plot(x, np.dot(X,model_w) + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

太好了！這次的擬合幾乎完美。請注意圖上方印出的 $\mathbf{w}$ 與 b 的數值：`w,b found by gradient descent: w: [1.], b: 0.0490`。梯度下降已經把原本的初始參數 $\mathbf{w},b$ 調整成 (1.0, 0.049)，也就是模型 $y = 1*x_0^2 + 0.049$，這跟我們原本的目標函數 $y = 1*x_0^2 + 1$ 已經非常接近。如果再多跑久一點，結果還有機會更貼近。 

### 特徵選擇（Selecting Features）
<a name='GDF'></a>
在上面的例子中，我們事先就知道需要一個 $x^2$ 項。但在現實中，往往不那麼容易一眼看出「應該加入哪些特徵」。我們可以先加入各種可能有用的特徵，再讓模型自己去「挑」出最重要的。例如，如果我們改用：$y = w_0x_0 + w_1x_1^2 + w_2x_2^3 + b$ 這樣的形式會怎樣呢？

請執行下面幾個程式區塊。 

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = x**2

# engineer features .
X = np.c_[x, x**2, x**3]   #<-- added engineered feature

In [ ]:
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha=1e-7)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("x, x**2, x**3 features")
plt.plot(x, X@model_w + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

請注意此時 $\mathbf{w}$ 的數值為 `[0.08 0.54 0.03]`，而 b 為 `0.0106`。這代表訓練後的模型為：
$$ 0.08x + 0.54x^2 + 0.03x^3 + 0.0106 $$
梯度下降透過放大 $w_1$（對應 $x^2$）相對於其他項的權重，來強調最能貼合資料的 $x^2$ 特徵。如果你持續訓練更久，其他項的影響會被進一步壓低。
> 梯度下降其實是在幫我們「挑出較正確的特徵」，方法就是加重與該特徵相關的參數。

我們來整理一下這個想法：
- 權重愈小，代表該特徵的重要性或「正確性」愈低；在極端情況下，若權重趨近於 0，表示這個特徵對擬合資料幾乎沒有幫助。
- 在上面的例子中，訓練完成後，與 $x^2$ 特徵相對應的權重遠大於 $x$ 或 $x^3$，因為它對擬合目標資料最有幫助。 

### 另一種觀點（An Alternate View）
上面我們是以「哪個多項式特徵比較符合目標資料」來選擇特徵。另一種思考方式是：一旦我們透過特徵工程產生了新特徵，本質上仍然是在做**線性迴歸**。在這個前提下，最好的特徵應該是「相對於目標值呈現線性關係」的特徵。以下透過一個例子來說明會更清楚。 

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = x**2

# engineer features .
X = np.c_[x, x**2, x**3]   #<-- added engineered feature
X_features = ['x','x^2','x^3']

In [ ]:
fig,ax=plt.subplots(1, 3, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X[:,i],y)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("y")
plt.show()

從上圖可以清楚看到，$x^2$ 這個特徵與目標值 $y$ 的關係是線性的。對於這樣的特徵，線性迴歸就能很輕鬆地學出一個良好的模型。

### 特徵縮放（Scaling features）
如同在上一個實驗中提到的，如果資料集中各特徵的尺度（數值範圍）差異很大，就應該進行特徵縮放，以加快梯度下降的收斂。在上面的例子中，$x$、$x^2$ 與 $x^3$ 的數值範圍自然差異極大。下面我們就對這個例子套用 Z-score 正規化。

In [ ]:
# create target data
x = np.arange(0,20,1)
X = np.c_[x, x**2, x**3]
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X,axis=0)}")

# add mean_normalization 
X = zscore_normalize_features(X)     
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X,axis=0)}")

現在，我們就可以試著使用更「大膽」一些的 alpha 值再跑一次梯度下降：

In [ ]:
x = np.arange(0,20,1)
y = x**2

X = np.c_[x, x**2, x**3]
X = zscore_normalize_features(X) 

model_w, model_b = run_gradient_descent_feng(X, y, iterations=100000, alpha=1e-1)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("Normalized x x**2, x**3 feature")
plt.plot(x,X@model_w + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

特徵縮放讓模型的收斂速度大幅提升。  
再看一次此時的 $\mathbf{w}$：其中 $w_1$（對應 $x^2$）是最被強調的，梯度下降幾乎把 $x^3$ 這一項的影響完全消除了。

### 複雜函數（Complex Functions）
透過特徵工程，即使是相當複雜的函數，我們也能建立出對應的模型：

In [ ]:
x = np.arange(0,20,1)
y = np.cos(x/2)

X = np.c_[x, x**2, x**3,x**4, x**5, x**6, x**7, x**8, x**9, x**10, x**11, x**12, x**13]
X = zscore_normalize_features(X) 

model_w,model_b = run_gradient_descent_feng(X, y, iterations=1000000, alpha = 1e-1)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("Normalized x x**2, x**3 feature")
plt.plot(x,X@model_w + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()



## 恭喜！（Congratulations）
在本實驗中，你已經：
- 學會如何利用特徵工程，讓線性迴歸也能擬合複雜、甚至高度非線性的函數
- 瞭解在進行特徵工程時，套用特徵縮放對於加速梯度下降收斂非常重要